In [19]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import webbrowser
from threading import Timer
# Sample data
# Dataset 1: Points for the first plot
df_points = pd.DataFrame({
    'id': [1, 2, 3],
    'x': [10, 20, 30],
    'y': [15, 25, 35]
})

# Dataset 2: Curves corresponding to each point in Dataset 1
curves = {
    1: pd.DataFrame({'x': [0, 1, 2], 'y': [0, 1, 0]}),
    2: pd.DataFrame({'x': [0, 1, 2], 'y': [0, 2, 0]}),
    3: pd.DataFrame({'x': [0, 1, 2], 'y': [0, 3, 0]})
}

# Initialize the Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div(
        dcc.Graph(
            id='scatter-plot',
            figure={
                'data': [
                    go.Scatter(
                        x=df_points['x'],
                        y=df_points['y'],
                        mode='markers',
                        marker={'size': 12},
                        text=df_points['id'],  # Hover text
                        hoverinfo='text'
                    )
                ],
                'layout': go.Layout(
                    title='Scatter Plot',
                    hovermode='closest'
                )
            }
        ),
        style={'width': '49%', 'display': 'inline-block'}
    ),
    html.Div(
        dcc.Graph(
            id='curve-plot',
            figure={
                'data': [],
                'layout': go.Layout(
                    title='Curve Plot'
                )
            }
        ),
        style={'width': '49%', 'display': 'inline-block'}
    )
])

@app.callback(
    Output('curve-plot', 'figure'),
    [Input('scatter-plot', 'hoverData')]
)
def display_curve(hoverData):
    if hoverData is None:
        return {'data': [], 'layout': go.Layout(title='Curve Plot')}
    
    point_id = hoverData['points'][0]['text']
    curve_data = curves.get(point_id)
    
    if curve_data is None:
        return {'data': [], 'layout': go.Layout(title='Curve Plot')}
    
    figure = {
        'data': [
            go.Scatter(
                x=curve_data['x'],
                y=curve_data['y'],
                mode='lines+markers'
            )
        ],
        'layout': go.Layout(
            title=f'Curve for Point ID {point_id}'
        )
    }
    return figure

def open_browser():
    webbrowser.open_new("http://localhost:8050/")

if __name__ == '__main__':
    Timer(1, open_browser).start()
    app.run_server(debug=True)


In [81]:
import scipy.io
import pandas as pd

# Load the .mat file
mat_data = scipy.io.loadmat('data/zl.mat')
ghat_data = scipy.io.loadmat('data/ghat.mat')

# Extract variables
x = mat_data['x'].flatten()
y = mat_data['y'].flatten()
z = mat_data['z'].flatten()
# ids = mat_data['id'].flatten()
t = ghat_data['t'].flatten()
ghat = ghat_data['ghat']
# Determine the number of rows in ghat
L = ghat.shape[0]

# Create sequential IDs from 1 to L
ids = range(1, L + 1)


# Create a DataFrame
df_3d = pd.DataFrame({'id': ids, 'x': x, 'y': y, 'z': z})

# Create a dictionary to hold the 2D curves
curves = {}

# Iterate over each id to create the corresponding 2D curve DataFrame
for i, id_val in enumerate(ids):
    curve_df = pd.DataFrame({'x': t, 'y': ghat[i, :]})
    curves[id_val] = curve_df






In [99]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

# Initialize the Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div(
        dcc.Graph(
            id='3d-scatter-plot',
            figure={
                'data': [
                    go.Scatter3d(
                        x=df_3d['x'],
                        y=df_3d['y'],
                        z=df_3d['z'],
                        mode='lines+markers',  # Display both lines and markers
                        marker={'size': 5},
                        line={'width': 2},  # Set the line width
                        text=df_3d['id'],  # Hover text
                        hoverinfo='text'
                    )
                ],
                'layout': go.Layout(
                    title='PC subspace',
                    hovermode='closest'
                )
            },
                style={'height': '600px'}

        ),
        style={'width': '60%', 'display': 'inline-block',  'vertical-align': 'top'}
    ),
    html.Div(
        dcc.Graph(
            id='2d-graph',
            figure={
                'data': [],
                'layout': go.Layout(
                    title='2D Graph'
                )
            },
            style={'height': '600px'}
        ),
        style={'width': '40%', 'display': 'inline-block', 'vertical-align': 'top'}
    )
])


In [100]:
@app.callback(
    Output('2d-graph', 'figure'),
    [Input('3d-scatter-plot', 'hoverData')]
)
def update_2d_graph(hoverData):
    if hoverData is None:
        return {'data': [], 'layout': go.Layout(title='2D Graph')}
    
    point_id = hoverData['points'][0]['text']
    curve_data = curves.get(point_id)
    
    if curve_data is None:
        return {'data': [], 'layout': go.Layout(title='2D Graph')}
    
    figure = {
        'data': [
            go.Scatter(
                x=curve_data['x'],
                y=curve_data['y'],
                mode='lines'
            )
        ],
        'layout': go.Layout(
            title=f'lam_{point_id+1}'
        )
    }
    return figure


In [101]:
import webbrowser
from threading import Timer


def open_browser():
    webbrowser.open_new("http://localhost:8050/")

if __name__ == '__main__':
    Timer(1, open_browser).start()
    app.run_server(debug=True)